## *Entrega do Plano de Projeto Mineração de Dados*
Uma grande loja de varejo forneceu um conjunto abrangente de dados históricos de vendas de três lojas, que inclui informações como data, produto, categoria, quantidade vendida, preço unitário e vendas totais. O objetivo deste projeto é extrair insights relevantes que possam auxiliar a gestão na tomada de decisões estratégicas – como otimização do mix de produtos, controle de estoque, definição de promoções e análise de desempenho financeiro.

### Bibliotecas usadas neste Notebook

In [1]:
import pandas as pd
import numpy as np

### 1.1 Importando os três datasets das lojas
Para carregar um dataset no formato csv, basta utilizar a função `read_csv` do pandas. Por padrão, ela considera _','_ como separador. Substituímos _','_ por _';'_ !


In [2]:
loja1 = pd.read_csv("Dataset_6_V_loja 01.csv", sep=";")
loja2 = pd.read_csv("Dataset_6_V_loja 02.csv", sep=";")
loja3 = pd.read_csv("Dataset_6_V_loja 03.csv", sep=";")

### 1.2 Conferindo as informações das três planilhas
Usando um laço for a gente nomeia cada dataset atribuindo eles aos seus respectivos dados

In [3]:
for df, name in [(loja1, 'Loja1'), (loja2, 'Loja2'), (loja3, 'Loja3')]:
    df["Loja"] = name

### 1.3 Concatenando todos os dataframes em um só
Atribuindo á váriavel 'dados' os três df das lojas usando uma função da biblioteca e reorganizando o índice deles com o argumento _'ignore_index=True'_


In [ ]:
dados = pd.concat([loja1, loja2, loja3], ignore_index=True)

### 1.4 Filtrando as colunas mais interessantes

In [ ]:
dados = dados[[
    "Data", "Produto", "Categoria",
    "Quantidade Vendida", "Preço Unitário",
    "Total de Vendas", "Loja"
]]


### 1.5 Verificando o total de linhas iniciais!


In [ ]:
LINHAS_INICIAIS = len(dados)
print(f"\n--- INÍCIO: {LINHAS_INICIAIS} linhas carregadas ---")


--- INÍCIO: 3000 linhas carregadas ---


### 1.6 Tratando as colunas numéricas

In [ ]:
cols_numericas = ["Quantidade Vendida", "Preço Unitário", "Total de Vendas"]

for col in cols_numericas:
    # Garante que é string, troca vírgula por ponto, converte para float
    dados[col] = dados[col].astype(str).str.replace(',', '.').apply(pd.to_numeric, errors='coerce')

### 1.7 Limpeza de Texto (Strip)
Remover espaços em branco extras é crucial para evitar que o Pandas conte o mesmo item duas vezes

In [ ]:
for col in ["Produto", "Categoria", "Loja"]:
    dados[col] = dados[col].str.strip().astype("string")


### 1.8 Corrigindo a DATA, no código anterior o programa eliminava cerca de 60% do DF só pelo formato das datas!

In [ ]:
# dayfirst=True é o segredo para datas brasileiras (DD/MM/AAAA)!!!
dados["Data"] = pd.to_datetime(dados["Data"], dayfirst=True, errors="coerce")

# Verifica perda
nulos_data = dados["Data"].isna().sum()
print(f"Linhas com data inválida (removidas): {nulos_data}")
dados = dados.dropna(subset=["Data"])

# Cria coluna Mês para a imputação
dados['Mes'] = dados['Data'].dt.month

Linhas com data inválida (removidas): 0


### 1.9 Imputação Condicional (primeira fase)
Esta parte evita a exclusão de linhas com dados faltando. Em vez de deletar, você preenche os buracos com o valor mais provável!

In [ ]:
print("Realizando Imputação...")

# Preenche vazios numéricos com a MEDIANA do grupo (Mês + Loja)
for col in cols_numericas:
    valor_imputacao = dados.groupby(['Mes', 'Loja'])[col].transform('median')
    dados[col].fillna(valor_imputacao, inplace=True)
    # Se sobrar algo, usa mediana geral
    dados[col].fillna(dados[col].median(), inplace=True)

# Preenche vazios de texto
for col in ["Produto", "Categoria"]:
    dados[col].fillna('Ausente', inplace=True)

Realizando Imputação...


C:\Users\megaf\AppData\Local\Temp\ipykernel_35076\1111022557.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dados[col].fillna(valor_imputacao, inplace=True)
C:\Users\megaf\AppData\Local\Temp\ipykernel_35076\1111022557.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

### 1.10 Remoção de Outliers com Z-Score
Em vez de decidir por conta própria que a quantidade máxima é 500, você deixa a estatística definir o que é um valor outlier (O Z-score é um cálculo que mede o quão longe um valor está da média)

In [ ]:
LIMITE_ZSCORE = 4 

print(f"Filtrando Outliers estatísticos (Z-Score > {LIMITE_ZSCORE})...")

for col in cols_numericas:
    # Calcula Z-Score
    media = dados[col].mean()
    std = dados[col].std()
    
    # Cria coluna temporária
    dados[f"ZScore_{col}"] = (dados[col] - media) / std
    
    # Filtra (mantém o que está dentro do limite)
    dados = dados.loc[
        (dados[f"ZScore_{col}"] >= -LIMITE_ZSCORE) & 
        (dados[f"ZScore_{col}"] <= LIMITE_ZSCORE)
    ]
    # Remove coluna temporária
    dados = dados.drop(columns=[f"ZScore_{col}"])


Filtrando Outliers estatísticos (Z-Score > 4)...


### 1.11 Filtro de Integridade

In [ ]:
dados = dados.loc[
    (dados["Preço Unitário"] > 0) & 
    (dados["Quantidade Vendida"] > 0) &
    (dados["Total de Vendas"] >= 0)
]

### 1.12 Finalização!



In [ ]:
dados = dados.drop(columns=['Mes'], errors='ignore')
dados = dados.reset_index(drop=True)

LINHAS_FINAIS = len(dados)
LINHAS_INICIAIS = 3000 # Valor fixo de referência
PORCENTAGEM_RETIDA = (LINHAS_FINAIS / LINHAS_INICIAIS) * 100

print("\n-------------------------------------------")
print("✅ RESULTADO FINAL (FASE PRÉ-PROCESSAMENTO)")
print("-------------------------------------------")
print(f"Linhas Originais: {LINHAS_INICIAIS}")
print(f"Linhas Finais:    {LINHAS_FINAIS}")
print(f"Aproveitamento:   {PORCENTAGEM_RETIDA:.2f}%")

print("\nEstrutura final (info):")
print(dados.info())

print("\nEstatísticas Descritivas (describe):")
print(dados.describe())

# Exporta o dataset tratado e limpo
dados.to_csv("dataset_tratado.csv", index=False) 
print("\nDados exportados para dataset_tratado.csv")


-------------------------------------------
✅ RESULTADO FINAL (FASE PRÉ-PROCESSAMENTO)
-------------------------------------------
Linhas Originais: 3000
Linhas Finais:    2996
Aproveitamento:   99.87%

Estrutura final (info):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2996 entries, 0 to 2995
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Data                2996 non-null   datetime64[ns]
 1   Produto             2996 non-null   string        
 2   Categoria           2996 non-null   string        
 3   Quantidade Vendida  2996 non-null   float64       
 4   Preço Unitário      2996 non-null   float64       
 5   Total de Vendas     2996 non-null   float64       
 6   Loja                2996 non-null   string        
dtypes: datetime64[ns](1), float64(3), string(3)
memory usage: 164.0 KB
None

Estatísticas Descritivas (describe):
                                Data  Quantidad